<a href="https://colab.research.google.com/github/awolaja/ChatGPT_Trading_Bot/blob/main/Yemitradingbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 Install all the packages through FinRL library

In [ ]:
!pip install setuptools==64.0.2
!apt-get install swig
!pip install wrds
!pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
swig is already the newest version (4.0.1-5build1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/AI4Finance-LLC/FinRL-Library.git to /tmp/pip-req-build-ugri09xh
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-LLC/FinRL-Library.git /tmp/pip-req-build-ugri09xh
  Resolved https://github.com/AI4Finance-LLC/FinRL-Library.git to commit 7414adb012b554f7c684d4615830dc5c31a094d1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning http

Installing additional packages needed are present, if not install them.

In [ ]:
!pip install Yahoo-finance
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install stockstats
!pip install OpenAi gym
!pip stable-baselines
!pip install tensorflow
!pip install pyfolio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: unknown command "stable-baselines"
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Import Packages

In [ ]:
import warnings
warnings.filterwarnings("ignore")


In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.config_tickers import DOW_30_TICKER
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent,DRLEnsembleAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline

from pprint import pprint

import sys
sys.path.append("../FinRL-Library")

import itertools

Create folders

In [ ]:
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)

check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

Download Data

Yahoo Finance is a website that provides stock data, financial news, financial reports, etc. All the data provided by Yahoo Finance is free.

FinRL uses a class YahooDownloader to fetch data from Yahoo Finance API
Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).
class YahooDownloader: Provides methods for retrieving daily stock data from Yahoo Finance API

Attributes
----------
    start_date : str
        start date of the data (modified from config.py)
    end_date : str
        end date of the data (modified from config.py)
    ticker_list : list
        a list of stock tickers (modified from config.py)

Methods
-------
fetch_data()
    Fetches data from yahoo API

In [ ]:
print(DOW_30_TICKER)


['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']


In [ ]:
TRAIN_START_DATE = '2009-04-01'
TRAIN_END_DATE = '2022-12-01'
TEST_START_DATE = '2021-01-01'
TEST_END_DATE = '2023-05-01'

df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TEST_END_DATE,
                     ticker_list = DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
df.head()


,date,open,high,low,close,volume,tic,day
0,2009-04-01,3.717500,3.892857,3.710357,3.299296,589372000,AAPL,2
1,2009-04-01,48.779999,48.930000,47.099998,35.911701,10850100,AMGN,2
2,2009-04-01,13.340000,14.640000,13.080000,11.688806,27701800,AXP,2
3,2009-04-01,34.520000,35.599998,34.209999,26.850754,9288800,BA,2
4,2009-04-01,27.500000,29.520000,27.440001,19.619818,15308300,CAT,2


In [ ]:
df.tail()


,date,open,high,low,close,volume,tic,day
103807,2023-04-28,488.690002,493.959991,488.119995,492.089996,2398400,UNH,4
103808,2023-04-28,228.649994,233.570007,228.059998,232.277145,11627900,V,4
103809,2023-04-28,38.560001,39.000000,38.459999,38.830002,19194400,VZ,4
103810,2023-04-28,35.139999,35.380001,34.959999,35.250000,11878500,WBA,4
103811,2023-04-28,151.289993,152.169998,150.179993,150.398682,4693900,WMT,4


In [ ]:
df.shape


(103812, 8)

In [ ]:
df.sort_values(['date','tic']).head()


,date,open,high,low,close,volume,tic,day
0,2009-04-01,3.717500,3.892857,3.710357,3.299296,589372000,AAPL,2
1,2009-04-01,48.779999,48.930000,47.099998,35.911701,10850100,AMGN,2
2,2009-04-01,13.340000,14.640000,13.080000,11.688806,27701800,AXP,2
3,2009-04-01,34.520000,35.599998,34.209999,26.850754,9288800,BA,2
4,2009-04-01,27.500000,29.520000,27.440001,19.619818,15308300,CAT,2


In [ ]:
len(df.tic.unique())


30

In [ ]:
df.tic.value_counts()


AAPL    3544
AMGN    3544
WMT     3544
WBA     3544
VZ      3544
V       3544
UNH     3544
TRV     3544
PG      3544
NKE     3544
MSFT    3544
MRK     3544
MMM     3544
MCD     3544
KO      3544
JPM     3544
JNJ     3544
INTC    3544
IBM     3544
HON     3544
HD      3544
GS      3544
DIS     3544
CVX     3544
CSCO    3544
CRM     3544
CAT     3544
BA      3544
AXP     3544
DOW     1036
Name: tic, dtype: int64

         Preprocess Data
Data preprocessing is a crucial step for training a high quality machine learning model. We need to check for missing data and do feature engineering in order to convert the data into a model-ready state.

Add technical indicators. In practical trading, various information needs to be taken into account, for example the historical stock prices, current holding shares, technical indicators, etc. In this article, we demonstrate two trend-following technical indicators: MACD and RSI.
Add turbulence index. Risk-aversion reflects whether an investor will choose to preserve the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial cris

In [ ]:
 INDICATORS = ['macd',
               'rsi_30',
               'cci_30',
               'dx_30']

The list I  provided above, INDICATORS, contains four different indicators commonly used in technical analysis for financial markets. Here's a brief explanation of each indicator:

MACD (Moving Average Convergence Divergence): MACD is a trend-following momentum indicator that calculates the difference between two moving averages of an asset's price. It consists of two lines: the MACD line (the difference between the 12-day and 26-day exponential moving averages) and the signal line (a 9-day exponential moving average of the MACD line). Traders often use MACD crossovers and divergences to identify potential trend reversals or confirm existing trends.
RSI (Relative Strength Index): RSI is a popular oscillator used to measure the speed and change of price movements. It ranges from 0 to 100 and is calculated based on the ratio of average upward price changes to average downward price changes over a specified period. Traders often interpret RSI values above 70 as overbought conditions (potential sell signal) and values below 30 as oversold conditions (potential buy signal).
CCI (Commodity Channel Index): CCI is a versatile indicator used to identify overbought and oversold levels, as well as potential trend reversals. It measures the relationship between an asset's price and its average price over a specific period, relative to its typical volatility. CCI values above +100 are considered overbought, while values below -100 are considered oversold.
DX (Directional Movement Index): DX is part of the larger Average Directional Index (ADX) indicator system, which helps assess the strength and direction of a trend. DX represents the strength of either an uptrend or a downtrend. Traders often use DX values above 25 to identify the presence of a strong trend and below 25 to indicate a weak or sideways market.
These indicators can be used in technical analysis to gain insights into price trends, momentum, and potential trading opportunities in financial markets. Different traders and analysts may use these indicators in combination with other tools and strategies to make informed trading decisions.

**bold text** code snippet you provided showcases the usage of a FeatureEngineer class for preprocessing data in a financial context.

In [ ]:
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(df)
processed = processed.copy()
processed = processed.fillna(0)
processed = processed.replace(np.inf,0)

Successfully added technical indicators
Successfully added turbulence index


In [ ]:
processed.sample(30)


,date,open,high,low,close,volume,tic,day,macd,rsi_30,cci_30,dx_30,turbulence
52231,2016-05-26,65.290001,65.370003,64.949997,58.765907,3593500,AXP,3,0.365450,57.214023,51.092423,21.151833,8.283045
34797,2014-01-06,48.770000,49.070000,48.200001,31.259056,16054800,VZ,0,-0.090838,48.500534,-48.235330,13.616492,15.376370
11269,2010-10-14,75.769997,77.089996,75.669998,54.344170,6616400,MCD,3,0.441361,62.420110,192.926188,21.645246,21.656329
47482,2015-10-01,174.399994,176.169998,173.039993,153.051239,3613000,GS,3,-4.013674,39.838517,-122.796726,32.494579,36.872757
49950,2016-02-03,118.537285,119.933075,116.912048,84.791977,6509781,IBM,2,-2.208521,40.167763,-75.807550,28.655213,73.959562
13114,2011-01-14,21.129999,21.240000,21.059999,14.831061,35176400,CSCO,4,0.192266,54.291207,127.784560,38.924502,28.970741
83739,2020-09-18,50.750000,51.160000,50.209999,46.774376,23816600,KO,4,0.791289,57.824558,83.238892,24.907676,10.850881
90958,2021-09-15,165.000000,165.960007,164.309998,158.992645,7935300,JNJ,2,-1.673908,41.963474,-221.487719,51.022595,16.097218
55408,2016-10-31,166.460007,166.539993,165.179993,133.800491,2289100,MMM,0,-1.985969,36.352892,-124.338731,31.861170,29.032825
51774,2016-05-04,161.199997,162.600006,159.089996,140.265930,3203900,GS,2,1.950255,50.448946,21.094362,6.662540,12.262287
